### Read data 

In [2]:
def data_tuple(file):

    data = []
    lst = []
    with open(file, encoding='utf-8', mode='r') as f:
        for line in f:
            if line == "\n":
              data.append(lst)
              lst = []
            
            else:
              lines = line.replace("\n",'').split(" ")
              lst.append(tuple(lines))
            
    return data


<br>
### Calculate emission parameters

In [3]:
# Write a function that estimates the emission parameters from the training set using MLE (maximum
# likelihood estimation):
# e(x|y) = Count(y -> x)/Count(y)
# with key <word, tag>

def emission_parameter(data, k=1):

    new_data = []
    new_tweets = []
  
    #word_count = {}
    tags = {} # count of appearance for each tag
    observations = {} # count of appearance for each observation
    smoothed = {} # count of observation appearance after smoothing
    match = {} # count of (X,Y) tuples
    
    # processing data with smoothing
    for lst in data:
        for wordTuple in lst:
            
            observation = wordTuple[0]
            tag = wordTuple[1]
                      
            observations[observation] = observations.get(observation,0) + 1
            tags[tag] = tags.get(tag,0) + 1

            if observations[observation] < k:
                smoothed['#UNK#'] = smoothed.get('#UNK#',0) + 1
                match[('#UNK#',tag)] = match.get(('#UNK#',tag),0) + 1         
            else:
                smoothed[observation] = smoothed.get(observation,0) + 1
                match[wordTuple] = match.get(wordTuple,0) + 1
    
    # compute emission parameter          
    em = {i: match[i]/tags[i[1]] for i in match}

    return list(observations), list(tags), em


<br>
### Calculate transition parameters 

$$p(y_{i}\;|\;y_{i-2},y_{i-1})=\frac{p(y_{i-2},y_{i-1},y_{i})}{p(y_{i-2},y_{i-1})}=\frac{Count(y_{i-2},y_{i-1},y_{i})}{Count(y_{i-2},y_{i-1})}$$

which could also be written as $$p(v\;|\;w,u)=\frac{Count(w,u,v)}{Count(w,u)}$$

where the state sequence is $w\rightarrow u\rightarrow v$

<br>

In [23]:
# function that estimates the transition parameters from the training set
# using MLE (maximum likelihood estimation)
# for any state v with previous two states w and u
# q(v|w,u) = Count(w,u,v)/Count(u,v)
# return: a dictionary with tuple of three states as key and their trans as value
# {(prev_previous, previous, current): transition probability}

def transition_parameter(data):
    double_transitions = {} #(u,v)
    triple_transitions = {} #(w,u,v)

    # processing data
    for lst in data:

        # y-1 = y0 = START
        prev_previous = ''
        previous = '#START#'
        current = '#START#'

        # starting from step 1 to step N
        for wordTuple in lst:
            
            tag = wordTuple[1]
            
            prev_previous = previous
            previous = current

            current = wordTuple[1]

            double_transitions[(prev_previous,previous)] = double_transitions.get((prev_previous,previous),0) + 1
            triple_transitions[(prev_previous,previous,current)] = triple_transitions.get((prev_previous,previous,current),0) + 1

        # final step N+1 --> 'STOP'
        double_transitions[(previous,current)] = double_transitions.get((previous,current),0) + 1
        double_transitions[(current,'#STOP#')] = double_transitions.get((current,'#STOP#'),0) + 1
        triple_transitions[(previous,current,'#STOP#')] = triple_transitions.get((previous,current,'#STOP#'),0) + 1
    
    trans = {i: triple_transitions[i]/double_transitions[i[:2]] for i in triple_transitions}

    return trans

transition_parameter(data_tuple('EN/train'))

{('#START#', '#START#', 'B-ADJP'): 0.009074410163339383,
 ('#START#', '#START#', 'B-ADVP'): 0.038112522686025406,
 ('#START#', '#START#', 'B-CONJP'): 0.0018148820326678765,
 ('#START#', '#START#', 'B-INTJ'): 0.0544464609800363,
 ('#START#', '#START#', 'B-NP'): 0.3466424682395644,
 ('#START#', '#START#', 'B-PP'): 0.014519056261343012,
 ('#START#', '#START#', 'B-SBAR'): 0.003629764065335753,
 ('#START#', '#START#', 'B-VP'): 0.11070780399274047,
 ('#START#', '#START#', 'O'): 0.42105263157894735,
 ('#START#', 'B-ADJP', 'B-ADVP'): 0.2,
 ('#START#', 'B-ADJP', 'I-ADJP'): 0.4,
 ('#START#', 'B-ADJP', 'O'): 0.4,
 ('#START#', 'B-ADVP', 'B-INTJ'): 0.047619047619047616,
 ('#START#', 'B-ADVP', 'B-NP'): 0.23809523809523808,
 ('#START#', 'B-ADVP', 'B-PP'): 0.047619047619047616,
 ('#START#', 'B-ADVP', 'B-VP'): 0.42857142857142855,
 ('#START#', 'B-ADVP', 'I-ADVP'): 0.19047619047619047,
 ('#START#', 'B-ADVP', 'O'): 0.047619047619047616,
 ('#START#', 'B-CONJP', 'I-CONJP'): 1.0,
 ('#START#', 'B-INTJ', 'B-A

<br>
### Viterbi Algorithm

* Base case:

$$\pi(0,u,START)=1$$

* Moving forward recursively: for any $j\in \left\{1,2,...,n-1\right\}$

$$\pi(j,y_{j},y_{j+1})=max_{y_{j-1}\in T}\;\pi(j-1,y_{j-1},y_{j})\cdot a(y_{j+1}|y_{j-1},y_{j})\cdot b(x_{j},y_{j})$$

* Final case:

$$\pi(n,y_{n},STOP)=max_{y_{n-1}\in T}\;\pi(n-1,y_{n-1},y_{n})\cdot a(STOP|y_{n-1},y_{n})\cdot b(x_{n},y_{n})$$

* Backtracking:

$$y_{n}^{*}=argmax_{y_{n}\in T}\pi(n,y_{n},STOP)$$

for any $j\in \left\{n-1,...,2,1\right\}$

$$y_{j}^{*}=argmax_{y_{j}\in T}\pi(j,y_{j},y_{j+1}^{*})$$

In [17]:
# Viterbi Algorithm
# w -> u -> v, u is the current state
# pi(j,u,v) = max{ pi(j-1,w,u) * a(v|w,u) * b(u,xj) } for all w in T
# u* = argmax {pi(j,u,v)} for all u in T
def viterbi(sentense, tags, observations, em, trans):

    # initialization of pi list
    # each item is a dictionary with tag as the key
    # and a distionary {next_tag: score} as the value
    pi = [{tag: {next_tag: 0.0 for next_tag in tags} for tag in tags} for i in range(len(sentense)-1)]
    pi.insert(0,{'#START#': {next_tag: 0.0 for next_tag in tags}})
    pi.append({tag: {'#STOP#': 0.0} for tag in tags})
    
    # j = 0
    # current state = START
    for v in tags: # loop for next state
        if ('#START#','#START#',v) not in trans: continue
            
        pi[0]['#START#'][v] = 1.0 * trans[('#START#','#START#',v)]

        
    # j = 1
    # pi(1,u,v)
    for v in tags: # loop for next state

            for u in tags: # loop for current state
                if ('#START#',u,v) not in trans: continue

                # emission parameter
                if sentense[0] in observations:
                    if (sentence[0],u) in em:
                        emission = em[(sentence[0],u)]
                    else: # emission not found
                        emission = 0.0
                else: # this word is #UNK#
                    if ('#UNK#',u) not in em: continue
                    emission = em[('#UNK#',u)]

                # maximum score
                pi[1][u][v] = pi[0]['#START#'][u] * trans[('#START#',u,v)] * emission

    # j = 2 ~ N-1
    for j in range(2, len(sentence)): # loop for each word

        for v in tags: # loop for next state

            for u in tags: # loop for current state

                # emission parameter
                if sentense[j-1] in observations:
                    if (sentence[j-1],u) in em:
                        emission = em[(sentence[j-1],u)]
                    else: # emission not found
                        emission = 0.0
                else: # this word is #UNK#
                    if ('#UNK#',u) not in em: continue
                    emission = em[('#UNK#',u)]

                for w in tags: # loop for previous state 
                    if (w,u,v) not in trans: continue
                        
                    # maximum score
                    score = pi[j-1][w][u] * trans[(w,u,v)] * emission
                    if score >= pi[j][u][v]:
                        pi[j][u][v] = score

    # j = N
    for u in tags:
        # emission parameter
        if sentense[-1] in observations:
            if (sentence[-1],u) in em:
                emission = em[(sentence[-1],u)]
            else: # emission not found
                emission = 0.0
        else: # this word is #UNK#
            if ('#UNK#',u) not in em: continue
            emission = em[('#UNK#',u)]

        for w in tags:
            if (w,u,'#STOP#') not in trans: continue
                
            # maximum score
            score = pi[-2][w][u] * trans[(w,u,'#STOP#')] * emission
            if score > pi[-1][u]['#STOP#']:
                pi[-1][u]['#STOP#'] = score
            

    # Backtracking

    prediction = [('',0.0)]
    
    # yn*
    for u in tags:
        score = pi[-1][u]['#STOP#']
        if score >= prediction[0][1]:
            prediction[0] = (u,score)

    # yn-1* ~ y1*
    for j in reversed(range(1,len(sentence))):
        v = prediction[0][0]
        prediction.insert(0, ('',0.0))

        for u in tags:
            score = pi[j][u][v]
            if score >= prediction[0][1]:
                prediction[0] = (u,score)

    # get sequence of tag predictioned
    sequence = []
    for t in prediction:
        sequence.append(str(t[0]))

    return sequence


<br>
### Prediction for EN 

In [24]:
if __name__ == '__main__':

    train = data_tuple('EN/train')

    test_in = data_tuple('EN/dev.in')

    observations, tags, em = emission_parameter(train)
    trans = transition_parameter(train)
    
    f = open('EN/dev.p4.out', encoding='utf-8', mode='w')

    for lst in test_in:        

        sentence = []

        for wordTuple in lst:
            sentence.append(wordTuple[0])

        prediction = viterbi(sentence,tags,observations,em,trans)

        for i in range(len(sentence)):
            if prediction:
                f.write('%s %s\n' % (sentence[i],prediction[i]))
            else: # prediction all zero
                f.write('%s O\n' % (sentence[i]))

        f.write('\n')

    print ('Writing to EN/dev.p4.out')
    f.close()

Writing to EN/dev.p4.out


<br>
### Prediction for FR 

In [26]:
if __name__ == '__main__':

    train = data_tuple('FR/train')

    test_in = data_tuple('FR/dev.in')

    observations, tags, em = emission_parameter(train)
    trans = transition_parameter(train)
    
    f = open('FR/dev.p4.out', encoding='utf-8', mode='w')

    for lst in test_in:        

        sentence = []

        for wordTuple in lst:
            sentence.append(wordTuple[0])

        prediction = viterbi(sentence,tags,observations,em,trans)

        for i in range(len(sentence)):
            if prediction:
                f.write('%s %s\n' % (sentence[i],prediction[i]))
            else: # prediction all zero
                f.write('%s O\n' % (sentence[i]))

        f.write('\n')

    print ('Writing to FR/dev.p4.out')
    f.close()

Writing to FR/dev.p4.out


<br>
### Evaluation 

In [20]:
# Evaluation

def get_entities(observed, sep=' ', output_col=1):

    example = 0
    word_index = 0
    entity = []
    last_ne = 'O'
    last_sent = ''
    last_entity = []

    observations = {}
    observations[example] = []

    for line in observed:
        line = line.strip()
        if line.startswith('##'):
            continue
        elif len(line) == 0:
            if entity:
                observations[example].append(list(entity))
                entity = []

            example += 1
            observations[example] = []
            word_index = 0
            last_ne = 'O'
            continue

        split_line = line.split(sep)
        value = split_line[output_col]
        ne = value[0]
        sent = value[2:]

        last_entity = []

        # check if it is start of entity
        if ne == 'B' or (ne == 'I' and last_ne == 'O') or \
                (ne == 'I' and last_ne != 'O' and last_sent != sent):
            if entity:
                last_entity = entity
            entity = [sent]
            entity.append(word_index)
        elif ne == 'I':
            entity.append(word_index)
        elif ne == 'O':
            if last_ne == 'B' or last_ne == 'I':
                last_entity = entity
            entity = []

        if last_entity:
            observations[example].append(list(last_entity))
            last_entity = []

        last_ne = ne
        last_sent = sent
        word_index += 1

    if entity:
        observations[example].append(list(entity))

    return observations


def compare_result(observed, predicted):
    
    total_observed = 0
    total_predicted = 0
    correct = {'Entities': 0, 'Sentiment': 0}

    for example in observed:
        observed_instance = observed[example]
        predicted_instance = predicted[example]
        total_observed += len(observed_instance)
        total_predicted += len(predicted_instance)

        for span in predicted_instance:
            span_sent = span[0]
            span_ne = (span[1], len(span) - 1)

            for observed_span in observed_instance:
                sent = observed_span[0]
                ne = (observed_span[1], len(observed_span) - 1)

                if span_ne == ne:
                    correct['Entities'] += 1
                    if span_sent == sent:
                        correct['Sentiment'] += 1

    print('Entities:')
    print('In gold-standard outputs: %d' % total_observed)
    print('In prediction outputs: %d' % total_predicted)
    print()
    for t in ('Entities', 'Sentiment'):
        prec = correct[t] / total_predicted
        recl = correct[t] / total_observed
        try:
            f = 2 * prec * recl / (prec + recl)
        except ZeroDivisionError:
            f = 0
        print('%s Correct: %d' % (t, correct[t]))
        print('%s F score: %.4f (Precision: %.4f, Recall: %.4f)' % (t, f, prec, recl))
        print()

In [27]:
for lan in ["EN", "FR"]:  
    with open(lan + '/dev.p4.out', encoding='utf-8', mode='r') as f:
        pred = get_entities(f)
    with open(lan + '/dev.out', encoding='utf-8', mode='r') as f:
        gold = get_entities(f)
        
    print(lan)
    compare_result(gold, pred)

EN
Entities:
In gold-standard outputs: 802
In prediction outputs: 110

Entities Correct: 30
Entities F score: 0.0658 (Precision: 0.2727, Recall: 0.0374)

Sentiment Correct: 29
Sentiment F score: 0.0636 (Precision: 0.2636, Recall: 0.0362)

FR
Entities:
In gold-standard outputs: 238
In prediction outputs: 210

Entities Correct: 49
Entities F score: 0.2187 (Precision: 0.2333, Recall: 0.2059)

Sentiment Correct: 30
Sentiment F score: 0.1339 (Precision: 0.1429, Recall: 0.1261)

